In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [30]:
#load the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
# Load the data
## Dropping irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [32]:
# Encode categorical variable
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [33]:
# OHE on Geography
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [34]:
one_hot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [35]:
geo_encoded_df=pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [36]:
# combile one hot encoded columns to data
data=pd.concat([data.drop('Geography', axis=1),geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [37]:
# save encoders in pickle file
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geo, file)

In [38]:
#divide dataset into dependent and independent features

x=data.drop(['Exited'], axis=1) #independent features except Exited
y=data['Exited']


In [39]:
# split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.2, random_state=42)

In [40]:
#apply scaler on data
scaler = StandardScaler()
#computes the mean and standard deviation of your x_train data.
x_train = scaler.fit_transform(x_train) 
# fits the scaler to the training data and then transforms it so that each feature has a mean of 0 and a standard deviation of 1.
x_test = scaler.transform(x_test)
#uses the same transformation parameters (from training) on the test set. This ensures your test data is scaled in the same way, avoiding data leakage.

In [41]:
x_train

array([[-0.38381126, -1.12244688,  2.60786056, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-0.73723819, -1.12244688,  0.12136034, ..., -0.99600797,
         1.74133801, -0.58350885],
       [ 0.32304261,  0.89091075, -0.54808203, ...,  1.00400803,
        -0.57427105, -0.58350885],
       ...,
       [ 0.85318301, -1.12244688, -0.06990891, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [ 0.14632915,  0.89091075,  0.40826421, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [ 0.45817644,  0.89091075,  1.1733412 , ..., -0.99600797,
         1.74133801, -0.58350885]])

In [42]:
x_test

array([[-0.59170945,  0.89091075, -0.64371665, ..., -0.99600797,
         1.74133801, -0.58350885],
       [-0.31104689,  0.89091075,  0.40826421, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-0.5397349 , -1.12244688,  0.50389883, ..., -0.99600797,
        -0.57427105,  1.71377008],
       ...,
       [ 0.63488991,  0.89091075, -0.54808203, ..., -0.99600797,
        -0.57427105,  1.71377008],
       [ 0.47896626,  0.89091075, -0.35681278, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-1.39211751,  0.89091075, -0.45244741, ...,  1.00400803,
        -0.57427105, -0.58350885]])

In [43]:
#will dump this into pickle file
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [44]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


# ANN Implimentation

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [46]:
data.shape[1]

13

In [47]:
## Build our ANN model
model=Sequential([
    Dense(64, activation = 'relu', input_shape=(x_train.shape[1],)), #HL1
    Dense(32, activation = 'relu'), #HL2
    Dense(1, activation = 'sigmoid') #output
]
)

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [49]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [50]:
##Compile moodel
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [51]:
## setup the tensorboard
log_dir='log/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S") #created log directectory 
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [52]:
##Set up earlystopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [53]:
## Train the model
history = model.fit(
    x_train,y_train, validation_data=(x_test,y_test), epochs = 100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
63/63 [==============================] - 3s 24ms/step - loss: 0.4626 - accuracy: 0.8070 - val_loss: 0.4238 - val_accuracy: 0.8204
Epoch 2/100
63/63 [==============================] - 1s 18ms/step - loss: 0.3995 - accuracy: 0.8390 - val_loss: 0.3966 - val_accuracy: 0.8347
Epoch 3/100
63/63 [==============================] - 1s 18ms/step - loss: 0.3680 - accuracy: 0.8430 - val_loss: 0.3708 - val_accuracy: 0.8472
Epoch 4/100
63/63 [==============================] - 1s 18ms/step - loss: 0.3480 - accuracy: 0.8605 - val_loss: 0.3882 - val_accuracy: 0.8346
Epoch 5/100
63/63 [==============================] - 1s 18ms/step - loss: 0.3485 - accuracy: 0.8595 - val_loss: 0.3677 - val_accuracy: 0.8480
Epoch 6/100
63/63 [==============================] - 1s 19ms/step - loss: 0.3357 - accuracy: 0.8650 - val_loss: 0.3679 - val_accuracy: 0.8539
Epoch 7/100
63/63 [==============================] - 1s 17ms/step - loss: 0.3321 - accuracy: 0.8515 - val_loss: 0.3728 - val_accuracy: 0.8484
Epoch 

In [54]:
##save model
model.save("model.h5")

c:\Users\palla\Downloads\Self_Study\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
## load tensorflow
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [56]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6006 (pid 20564), started 4:03:07 ago. (Use '!kill 20564' to kill it.)